In [3]:
# gsioc defines a class used for controlling gsioc devices
# ID is an integer
# immediate commands are an ascii string
# buffered commands are an ascii string

import datetime, time
import serial
import binascii

class gsioc:
    def __init__(self,serial=None):
        self.serial = serial

    def createSerial(self,port=0,timeout=0.1):
        self.port = port
        self.timeout = timeout
        # Initiate serial connection
        s = serial.Serial(port)
        #s = serial.serial_for_url("loop://")  # loop for testing
        s.baudrate = 9600
        s.bytesize = 8
        s.parity = serial.PARITY_NONE
        s.stopbits = 1
        s.timeout = timeout
        self.serial = s
        try :
            s.open()
            print(s)
        except :
            print(s)

    def closeSerial(self):
        self.serial.close()

    def connect(self,ID=0):
        if( int(ID) not in range(64) ):
            raise Exception("ID out of range [0,63]")
        ID += 128
        s = self.serial
        s.flushInput()
        s.write(bytes.fromhex('ff'))
        time.sleep(self.timeout)   # Passively wait for all devices to disconnect
        s.write(ID.to_bytes(1,byteorder='big'))
        resp = s.read(1)    # Will raise serialTimoutException after timeout
        print(str(datetime.datetime.now()) + " -- Connected to device ", ID-128)

    # returns byte array
    # Use str(resp,'ascii') or resp.decode('ascii') to get ascii string
    def iCommand(self,commandstring):
        command = binascii.a2b_qp(commandstring)
        if(command[0] not in range(0,255)):     # Change this to correct range
            raise Exception("Command out of range")
        s = self.serial
        s.flushInput()
        s.write(command[0:1])
        resp = bytearray(0)
        while(True):
            resp.append(s.read(1)[0])
            if(resp[len(resp)-1] > 127):
                resp[len(resp)-1] -= 128
                # print(str(datetime.datetime.now()) + " -- Immediate response complete")
                break
            else:
                s.write(bytes.fromhex("06"))
        return resp.decode("ascii")

    def bCommand(self,commandstring):
        data = binascii.a2b_qp("\n" + commandstring + "\r")
        s = self.serial
        s.flushInput()
        resp = bytearray(0)

        # begin buffered command by sending \n until the device echos \n or times out
        firstErrorPrinted = False # This is used to prevent repetitive printing 
        # begin loop
        while(True):
            s.write(data[0:1])    # send line feed
            readySig = s.read(1)[0]
            if(readySig == 10):
                print(str(datetime.datetime.now()) + " -- Starting Buffered command")
                break
            elif(readySig == 35):
                if(not firstErrorPrinted):
                    print("Device busy. Waiting...")
                    firstErrorPrinted = True
            else:
                raise Exception("Did not recieve \\n (0x0A) or # as response")
        resp.append(readySig)

        # Send buffered data
        for i in range(1,len(data)):
            s.write(data[i:i+1])
            resp.append(s.read(1)[0])
            if( resp[i] != data[i] ):
                raise Exception("Recieved " + str(resp,'ascii') + " instead of " + str(data[i:i+1]))
            if( resp[i] == 13 ):
                print(str(datetime.datetime.now()) + " -- Buffered command complete")
                return resp

        # This will happen if sending the data failed
        print(str(datetime.datetime.now()) + " -- Buffered command FAILED")
        resp_no_whitespace = resp[1:len(resp)-2]
        return resp_no_whitespace.decode("ascii")


In [4]:
import time

In [5]:
connection_string = 'W7= Edit        Manual   Run               '

In [7]:
h = gsioc()
h.createSerial(port='COM29',timeout=0.1)

Serial<id=0x222d3b6cc50, open=True>(port='COM29', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=0.1, xonxoff=False, rtscts=False, dsrdtr=False)


In [8]:
h.connect(ID=4)

2017-12-06 10:21:56.909254 -- Connected to device  4


In [386]:
# check if we started up correctly
def checkConnection():
    display = []
    is_connected = False
    for i in range(9):
        if h.iCommand("w") == connection_string:
            is_connected = True
            break
    return is_connected         
        

In [387]:
connected = checkConnection()

In [388]:
connected

True

In [278]:
# start priming
h.bCommand("K=c|")
time.sleep(0.1)
h.bCommand("K=c|")
time.sleep(0.1)
h.bCommand("K=a|")

2017-11-29 17:40:27.729309 -- Starting Buffered command
2017-11-29 17:40:27.779411 -- Buffered command complete
2017-11-29 17:40:27.895812 -- Starting Buffered command
2017-11-29 17:40:27.950596 -- Buffered command complete
2017-11-29 17:40:28.077238 -- Starting Buffered command
2017-11-29 17:40:28.115122 -- Buffered command complete


bytearray(b'\nK=a|\r')

In [279]:
# stop priming
h.bCommand("K=a|")
time.sleep(0.2)
h.bCommand("K=X|")
time.sleep(0.2)
h.bCommand("K=X|")
time.sleep(0.2)
h.bCommand("K")

2017-11-29 17:40:33.216217 -- Starting Buffered command
2017-11-29 17:40:33.265422 -- Buffered command complete
2017-11-29 17:40:33.477853 -- Starting Buffered command
2017-11-29 17:40:33.521452 -- Buffered command complete
2017-11-29 17:40:33.732115 -- Starting Buffered command
2017-11-29 17:40:33.784661 -- Buffered command complete
2017-11-29 17:40:33.993786 -- Starting Buffered command
2017-11-29 17:40:34.006988 -- Buffered command complete


bytearray(b'\nK\r')

In [389]:
flow_rate = 2.0

In [390]:
# start pumping
h.bCommand("K=c|")
time.sleep(0.1)
h.bCommand("K=a|")
time.sleep(0.1)
h.bCommand("K=0|")
time.sleep(0.1)
h.bCommand("K=E|")
time.sleep(0.1)
for char in str(flow_rate):
    h.bCommand("K=" + char + "|")
    time.sleep(0.1)
h.bCommand("K=E|")
time.sleep(0.1)
h.bCommand("K=d|")

2017-11-30 16:27:42.740213 -- Starting Buffered command
2017-11-30 16:27:42.790861 -- Buffered command complete
2017-11-30 16:27:42.900884 -- Starting Buffered command
2017-11-30 16:27:42.951486 -- Buffered command complete
2017-11-30 16:27:43.062324 -- Starting Buffered command
2017-11-30 16:27:43.111690 -- Buffered command complete
2017-11-30 16:27:43.222625 -- Starting Buffered command
2017-11-30 16:27:43.272613 -- Buffered command complete
2017-11-30 16:27:43.382728 -- Starting Buffered command
2017-11-30 16:27:43.431951 -- Buffered command complete
2017-11-30 16:27:43.542220 -- Starting Buffered command
2017-11-30 16:27:43.591980 -- Buffered command complete
2017-11-30 16:27:43.702674 -- Starting Buffered command
2017-11-30 16:27:43.751905 -- Buffered command complete
2017-11-30 16:27:43.862737 -- Starting Buffered command
2017-11-30 16:27:43.913015 -- Buffered command complete
2017-11-30 16:27:44.024866 -- Starting Buffered command
2017-11-30 16:27:44.075861 -- Buffered command c

bytearray(b'\nK=d|\r')

In [319]:
# stop pumping
h.bCommand("K=e|")
time.sleep(1)
h.bCommand("K=X|")
time.sleep(0.5)
h.bCommand("K=X|")
time.sleep(0.2)
h.bCommand("K")

2017-11-29 17:48:36.496632 -- Starting Buffered command
2017-11-29 17:48:36.547014 -- Buffered command complete
2017-11-29 17:48:37.548405 -- Starting Buffered command
2017-11-29 17:48:37.601728 -- Buffered command complete
2017-11-29 17:48:38.107861 -- Starting Buffered command
2017-11-29 17:48:38.154671 -- Buffered command complete
2017-11-29 17:48:38.364962 -- Starting Buffered command
2017-11-29 17:48:38.385722 -- Buffered command complete


bytearray(b'\nK\r')

In [329]:
# start pumping
h.bCommand("K=ca0E|")


2017-11-29 17:52:47.713025 -- Starting Buffered command


IndexError: index out of range

In [323]:
h.bCommand("K=e|")

IndexError: index out of range

In [301]:
h.bCommand("K=X|")
time.sleep(0.02)
h.bCommand("K=X|")
time.sleep(0.02)
h.bCommand("K")

2017-11-29 17:44:58.807035 -- Starting Buffered command
2017-11-29 17:44:58.855575 -- Buffered command complete
2017-11-29 17:44:58.892502 -- Starting Buffered command
2017-11-29 17:44:58.941330 -- Buffered command complete
2017-11-29 17:44:58.972587 -- Starting Buffered command
2017-11-29 17:44:58.992664 -- Buffered command complete


bytearray(b'\nK\r')

In [391]:
h.bCommand("K")

2017-11-30 16:29:32.552228 -- Starting Buffered command
2017-11-30 16:29:32.571279 -- Buffered command complete


bytearray(b'\nK\r')

In [303]:
h.bCommand("K=X|")
time.sleep(0.02)

2017-11-29 17:45:19.405758 -- Starting Buffered command
2017-11-29 17:45:19.455931 -- Buffered command complete


In [89]:
# disconnect pump heads
h.bCommand("aN")

2017-11-29 15:41:29.436503 -- Starting Buffered command
2017-11-29 15:41:29.467052 -- Buffered command complete


bytearray(b'\naN\r')

In [37]:
h.bCommand("?R1")

IndexError: index out of range

In [36]:
h.bCommand("?P")

IndexError: index out of range

In [34]:
h.bCommand("?S")

2017-12-06 10:35:22.175554 -- Starting Buffered command
2017-12-06 10:35:22.205698 -- Buffered command complete


bytearray(b'\n?S\r')

In [369]:
h.iCommand("?")

'R----'

In [334]:
h.iCommand("$")

'$'

In [377]:
# start pumping
h.bCommand("K=e|")
time.sleep(1)
h.bCommand("K")

2017-11-29 18:17:02.829970 -- Starting Buffered command
2017-11-29 18:17:02.880067 -- Buffered command complete
2017-11-29 18:17:03.884001 -- Starting Buffered command
2017-11-29 18:17:03.899628 -- Buffered command complete


bytearray(b'\nK\r')

In [363]:
h.bCommand("K=e|")

2017-11-29 18:07:21.011301 -- Starting Buffered command
2017-11-29 18:07:21.059430 -- Buffered command complete


bytearray(b'\nK=e|\r')

In [374]:
h.bCommand("E1=2")

2017-11-29 18:14:26.383449 -- Starting Buffered command
2017-11-29 18:14:26.433589 -- Buffered command complete


bytearray(b'\nE1=2\r')

In [375]:
h.bCommand("G,0,10")

2017-11-29 18:15:57.452605 -- Starting Buffered command
2017-11-29 18:15:57.521938 -- Buffered command complete


bytearray(b'\nG,0,10\r')

In [378]:
h.iCommand("Q")

'p0.0'